In [ ]:
!git clone https://github.com/Yasgant/project-nmnbns.git
!cp ./project-nmnbns/game/* ./ -rf
!pip install pygame
!mkdir replay

Cloning into 'project-nmnbns'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 107 (delta 58), reused 76 (delta 30), pack-reused 0
Receiving objects: 100% (107/107), 73.29 KiB | 8.14 MiB/s, done.
Resolving deltas: 100% (58/58), done.
     |████████████████████████████████| 21.8 MB 1.2 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
action_size = 9
stack_size = 2
learning_rate = 0.0025
state_size = (101, 121, stack_size)
total_episodes = 1000
batch_size = 64
gamma = 0.95
remember_size = 100000
pre_train = 5
epsilon = 0.1
new_img_size = (50, 30)

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import deque
import random
import sys
sys.path.append('/content')
from core import *
from danmaku import *
from enemies import *
from tensorflow.keras import layers, optimizers, models
import copy
from collections import deque

pygame 2.1.2 (SDL 2.0.16, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
class MyModel:
    def __init__(self, state_size = state_size, action_size = action_size, learning_rate = learning_rate):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.model = models.Sequential([
            layers.Input(shape = (*new_img_size, stack_size)),
            layers.Conv2D(filters = 32, kernel_size = 4),
            layers.Conv2D(filters = 64, kernel_size = 2),
            layers.Flatten(),
            #layers.Dense(2048, activation = 'relu'),
            #layers.Dense(2048, activation = 'relu'),
            #layers.Dense(512, activation = 'relu'),
            #layers.Dense(512, activation = 'relu'),
            #layers.Dense(128, activation = 'relu'),
            layers.Dense(512, activation = 'relu'),
            layers.Dense(64, activation = 'relu'),
            layers.Dense(action_size, activation = 'linear')
        ])
        self.model.compile(loss = 'mean_squared_error', optimizer = optimizers.Adam(learning_rate))
    
    def predict(self, img, eps = epsilon):
        if np.random.random() < eps:
            return np.random.randint(self.action_size)
        return np.argmax(np.array(self.model(np.array([img])))[0])
    
    def predicts(self, imgs):
        return np.array(self.model(imgs))

In [ ]:
stacked_imgs = deque([np.zeros(new_img_size) for i in range(stack_size)], maxlen = stack_size)

def stack_img(stacked_imgs, img, fir = False):
    stacked_imgs.append(img)
    if fir:
        for i in range(stack_size - 1):
            stacked_imgs.append(img)
    stacked_img = np.stack(stacked_imgs, axis = 2)
    return stacked_img, stacked_imgs

In [ ]:
class Memory:
    def __init__(self, maxlen = remember_size):
        self.queue = deque(maxlen = maxlen)
    
    def remember(self, img):
        self.queue.append(img)
    
    def sample(self, size = batch_size):
        return random.sample(self.queue, size)
    

In [ ]:
def proc(img):
    new_img = np.zeros(new_img_size)
    x, y = 0, 0
    flag = False
    for i in range(len(img)):
        for j in range(len(img[0])):
            if img[i][j] == 0.3:
                x, y = i, j
                flag = True
                break
        if flag:
            break
    def in_map(x, y):
        return 0 <= x < state_size[0] and 0 <= y < state_size[1]
    for dx in range(-25, 25):
        for dy in range(-25, 5):
            if in_map(x+dx, y+dy):
                new_img[25+dx, 25+dy] = img[x+dx, y+dy]
            else:
                new_img[25+dx, 25+dy] = 1
    return new_img

In [ ]:
data = StageData.empty('S1',10*60*60)
# 0 ~ 1000
data.insert(0, AimmingEnemy3(100, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))
data.insert(9, AimmingEnemy3(110, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))
data.insert(18, AimmingEnemy3(120, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))
data.insert(27, AimmingEnemy3(130, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))
data.insert(36, AimmingEnemy3(140, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))
data.insert(45, AimmingEnemy3(150, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))
data.insert(54, AimmingEnemy3(160, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))
data.insert(63, AimmingEnemy3(170, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))
data.insert(72, AimmingEnemy3(180, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))
data.insert(81, AimmingEnemy3(190, 100, shoot_time = 10, bullet_speed=10, arc = np.pi / 24, alive_time=5*60))

# 1000 ~ 1500
data.insert(1000, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1020, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1040, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1060, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1080, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1100, AimmingEnemy(100, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1000, AimmingEnemy(300, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1020, AimmingEnemy(300, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1040, AimmingEnemy(300, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1060, AimmingEnemy(300, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1080, AimmingEnemy(300, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))
data.insert(1100, AimmingEnemy(300, 10, shoot_time = 2, bullet_speed=10, alive_time=3*60, vy = 3))

# 1500 ~ 2500
data.insert(1500, RandomEnemy(200, 100, shoot_time = 5, bullet_speed=5, alive_time=7*60))

# 2500 ~ 3500
data.insert(2500, RandomEnemyWithFall(200, 100, shoot_time = 5, bullet_speed=5, alive_time=7*60))

In [ ]:
#pretrain
memory = Memory()
for i in range(pre_train):
    G = game_with_op(Player(320, 400), copy.deepcopy(data))
    flag = True
    while True:
        if flag:
            img = G.get_img()
            img = proc(img)
            img, stacked_imgs = stack_img(stacked_imgs, img, True)
            flag = False
        action = np.random.randint(action_size)
        reward, done = G.op(action)
        next_img = G.get_img()
        next_img = proc(next_img)
        next_img, stacked_imgs = stack_img(stacked_imgs, next_img)
        if done:
            next_img = np.zeros((*new_img_size, stack_size))
            memory.remember((img, action, reward, next_img))
            break
        memory.remember((img, action, reward, next_img))
        img = next_img

In [ ]:
#train
model = MyModel()
for j in range(total_episodes):
    with open('./replay/'+str(j)+'.txt', "w") as f:
        G = game_with_op(Player(320, 400), copy.deepcopy(data))
        flag = True
        total_reward = 0
        while True:
            if flag:
                img = G.get_img()
                img = proc(img)
                img, stacked_imgs = stack_img(stacked_imgs, img, True)
                flag = False
            action = model.predict(img)
            f.write(str(action)+'\n')
            reward, done = G.op(action)
            total_reward += reward
            #print('Frame: {}, action: {}, reward: {:.0f}'.format(G.frame, action, total_reward))
            #print('X: {:.1f} Y: {:.1f}'.format(G.player.x, G.player.y))
            if done:
                next_img = np.zeros(new_img_size)
                next_img, stacked_imgs = stack_img(stacked_imgs, next_img)
                memory.remember((img, action, reward, next_img))
                print("Episode {} finished! Frames: {} Reward: {}".format(j, G.frame, total_reward))
            else:
                next_img = G.get_img()
                next_img = proc(next_img)
                #while (next_img != 0.7).all():
                    #G.op(0)
                    #f.write('0\n')
                    #print('skipping')
                    #next_img = G.get_img()
                next_img, stacked_imgs = stack_img(stacked_imgs, next_img)
                memory.remember((img, action, reward, next_img))
                img = next_img
            
            batch = memory.sample()
            img_batch = np.array([each[0] for each in batch], ndmin=3)
            action_batch = np.array([each[1] for each in batch])
            reward_batch = np.array([each[2] for each in batch]) 
            next_img_batch = np.array([each[3] for each in batch], ndmin=3)

            next_img_batch_Q = model.predicts(next_img_batch)
            img_batch_Q = model.predicts(img_batch)

            for i in range(batch_size):
                img_batch_Q[i][action_batch[i]] = (1 - 0.1) * img_batch_Q[i][action_batch[i]] + \
                0.1 * (reward_batch[i] + gamma * np.amax(next_img_batch_Q[i]))
            model.model.fit(img_batch, img_batch_Q, verbose = 0)
            if done:
                if j % 20 == 19:
                    model.model.save('./drive/MyDrive/models/')
                break



Episode 0 finished! Frames: 125 Reward: -76577.4334505121
Episode 1 finished! Frames: 164 Reward: -69701.83682688663
Episode 2 finished! Frames: 86 Reward: -83359.46443605114
Episode 3 finished! Frames: 56 Reward: -89067.71876304272
Episode 4 finished! Frames: 93 Reward: -82225.71131911801
Episode 5 finished! Frames: 71 Reward: -86215.72045924274
Episode 6 finished! Frames: 86 Reward: -83468.19538484165
Episode 7 finished! Frames: 110 Reward: -79117.75286446477
Episode 8 finished! Frames: 123 Reward: -76982.34304272276
Episode 9 finished! Frames: 117 Reward: -78054.22184875226
Episode 10 finished! Frames: 114 Reward: -78415.82176474224
Episode 11 finished! Frames: 130 Reward: -75768.14674911773
Episode 12 finished! Frames: 65 Reward: -87348.63990873026
Episode 13 finished! Frames: 47 Reward: -90817.62515451833
Episode 14 finished! Frames: 56 Reward: -89065.77633152537
Episode 15 finished! Frames: 86 Reward: -83401.38517172038
Episode 16 finished! Frames: 85 Reward: -83645.33286480223
E

In [ ]:
model.model.save('content/drive/MyDrive/model/copy1')

INFO:tensorflow:Assets written to: content/drive/MyDrive/model/copy1/assets


In [ ]:
#train2
model = MyModel()
for j in range(total_episodes):
    with open('./replay/'+str(j)+'.txt', "w") as f:
        G = game_with_op(Player(320, 400), copy.deepcopy(data))
        flag = True
        total_reward = 0
        while True:
            if flag:
                img = G.get_img()
                img, stacked_imgs = stack_img(stacked_imgs, img, True)
                flag = False
            action = model.predict(img)
            f.write(str(action)+'\n')
            reward, done = G.op(action)
            total_reward += reward
            #print('Frame: {}, action: {}, reward: {:.0f}'.format(G.frame, action, total_reward))
            #print('X: {:.1f} Y: {:.1f}'.format(G.player.x, G.player.y))
            if done:
                next_img = np.zeros(state_size[:2])
                next_img, stacked_imgs = stack_img(stacked_imgs, next_img)
                memory.remember((img, action, reward, next_img))
                print("Episode {} finished! Frames: {} Reward: {}".format(j, G.frame, total_reward))
            else:
                next_img = G.get_img()
                while (next_img != 0.7).all():
                    G.op(0)
                    f.write('0\n')
                    print('skipping')
                    next_img = G.get_img()
                next_img, stacked_imgs = stack_img(stacked_imgs, next_img)
                memory.remember((img, action, reward, next_img))
                img = next_img
            

            if done:
                for cc in range(10):
                    batch = memory.sample(50 * batch_size)
                    img_batch = np.array([each[0] for each in batch], ndmin=3)
                    action_batch = np.array([each[1] for each in batch])
                    reward_batch = np.array([each[2] for each in batch]) 
                    next_img_batch = np.array([each[3] for each in batch], ndmin=3)

                    next_img_batch_Q = model.predicts(next_img_batch)
                    img_batch_Q = model.predicts(img_batch)
                    for i in range(batch_size):
                        img_batch_Q[i][action_batch[i]] = (1 - 0.1) * img_batch_Q[i][action_batch[i]] + \
                        0.1 * (reward_batch[i] + gamma * np.amax(next_img_batch_Q[i]))
                    model.model.fit(img_batch, img_batch_Q, verbose = 0)
                if j % 20 == 0:
                    model.model.save('./drive/MyDrive/models/')
                break

